In [1]:
import os
import sys
import re
import random
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import subprocess

from sklearn.cluster import KMeans
import sklearn.metrics as sm
%matplotlib inline
pd.options.display.max_colwidth = 1000

sys.path.insert(0,'/home/lhtran/sc/')
from get_rmsds_functions import TMalign
from get_rmsds_functions import get_RMSD
import check_interactions_to_lig
import get_pock_res_return as pockres
from libSlurm import make_submit_file
import libPDB

from tqdm import tqdm



In [2]:
home_dir = '/home/lhtran/class/cheme_577/challenge_5'
os.chdir(home_dir)
pdb = f'{home_dir}/5tje_trim.pdb'
fasta = f'{home_dir}/5tje_trim.fasta'

### superfold predictions of complexes

In [3]:
pdb = f'{home_dir}/5tje_trim.pdb'

In [17]:
cmds = []

#tag = pdb.strip().split('/')[-1].split('.pdb')[0]
outdir = f'{home_dir}/output_multimer/'    
os.makedirs(outdir, exist_ok=True)

cmd = f'/software/lab/superfold/superfold {pdb}'
cmd += ' --models all '
cmd += '--version multimer '
cmd += '--type multimer_v2 '
cmd += '--max_recycles 5 '
cmd += '--recycle_tol 0 '
cmd += '--seed_start 0 '
cmd += '--nstruct 1 '
cmd += f'--out_dir {outdir} '
cmd += '--output_pae '
cmd += '--output_summary '
cmd += '--simple_rmsd '
cmd += '--initial_guess'
cmds.append(cmd)


In [18]:
cmd

'/software/lab/superfold/superfold /home/lhtran/class/cheme_577/challenge_5/5tje_trim.pdb --models all --version multimer --type multimer_v2 --max_recycles 5 --recycle_tol 0 --seed_start 0 --nstruct 1 --out_dir /home/lhtran/class/cheme_577/challenge_5/output_multimer/ --output_pae --output_summary --simple_rmsd --initial_guess'

In [19]:
with open(f'{home_dir}/superfold_cmd_multi', 'w') as f:
    f.write('\n'.join(cmds))


In [20]:
make_submit_file(cmds = f'{home_dir}/superfold_cmd_multi', submitfile = f'{home_dir}/submit_superfold_multi.sh', time='3:00:00', group_size=1, logsfolder='logs', cpus=2, mem='8g', needs_gpu=True, gpu_type='rtx2080')

### mlfold

In [6]:
cmd = f'/software/mlfold/mlfold.py --pdb-file {pdb} --af-models 1 2 3 4 5 --output-dir {home_dir}/output_af2/'

In [ ]:
with open(f'{home_dir}/af2_cmd', 'w') as f:
    f.write(cmd)


In [7]:
cmd

'/software/mlfold/mlfold.py --pdb-file /home/lhtran/class/cheme_577/challenge_5/5tje_trim.pdb --af-models 1 2 3 4 5 --output-dir /home/lhtran/class/cheme_577/challenge_5/output_af2/'

### colabFold

In [21]:
cmds = []
cmds.append(f'/software/containers/colabfold.sif colabfold_batch --model-type alphafold2_multimer_v3 {fasta} {home_dir}/colabfold_multi/')
cmds.append(f'/software/containers/colabfold.sif colabfold_batch --model-type alphafold2_multimer_v3 {fasta} {home_dir}/colabfold_multi_template/ --templates')
cmds.append(f'/software/containers/colabfold.sif colabfold_batch --model-type alphafold2_ptm {fasta} {home_dir}/colabfold_mono/')
cmds.append(f'/software/containers/colabfold.sif colabfold_batch --model-type alphafold2_ptm {fasta} {home_dir}/colabfold_mono_template/ --templates')


In [22]:
cmds[-1]

'/software/containers/colabfold.sif colabfold_batch --model-type alphafold2_ptm /home/lhtran/class/cheme_577/challenge_5/5tje_trim.fasta /home/lhtran/class/cheme_577/challenge_5/colabfold_mono_template/ --templates'

### collect results

In [ ]:
#mono
rmds_A = 0.516
rmsd_H = 0.433
rmsd_G = 0.516
rmsd_I = 5.596